# Regras de Associação no Titanic com mlxtend

**Objetivo:** descobrir perfis de passageiros com maior (ou menor) chance de sobrevivência usando regras de associação.

- Dataset: Titanic (`train.csv` do Kaggle ou equivalente)
- Biblioteca: `mlxtend.frequent_patterns` (Apriori + association_rules)


Caio Passos de Lima Albuquerque - 536506

In [4]:
import warnings

# Suprime especificamente o DeprecationWarning relacionado a datetime.utcnow
warnings.filterwarnings("ignore", category=DeprecationWarning)

print("Avisos de 'datetime.utcnow()' serão suprimidos a partir de agora.")

Avisos de 'datetime.utcnow()' serão suprimidos a partir de agora.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [2]:
!pip install mlxtend

In [3]:
# 1. Imports básicos
import pandas as pd
import numpy as np

from mlxtend.frequent_patterns import apriori, association_rules

# Configurações de exibição
pd.set_option('display.max_columns', 100)
pd.set_option('display.precision', 3)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [6]:
# 2. Carregando os dados
# TODO: ajustar o caminho do arquivo train.csv

import pandas as pd

caminho_arquivo = 'train.csv'
titanic = pd.read_csv(caminho_arquivo)
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.050,NaN,S


In [7]:
# 3.1. Inspecionar estrutura
titanic.info()
titanic.describe(include='all')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000,891.000,891.000,891,891,714.000,891.000,891.000,891,891.000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Dooley, Mr. Patrick",male,NaN,NaN,NaN,347082,NaN,G6,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000,0.384,2.309,NaN,NaN,29.699,0.523,0.382,NaN,32.204,NaN,NaN
std,257.354,0.487,0.836,NaN,NaN,14.526,1.103,0.806,NaN,49.693,NaN,NaN
min,1.000,0.000,1.000,NaN,NaN,0.420,0.000,0.000,NaN,0.000,NaN,NaN
25%,223.500,0.000,2.000,NaN,NaN,20.125,0.000,0.000,NaN,7.910,NaN,NaN
50%,446.000,0.000,3.000,NaN,NaN,28.000,0.000,0.000,NaN,14.454,NaN,NaN
75%,668.500,1.000,3.000,NaN,NaN,38.000,1.000,0.000,NaN,31.000,NaN,NaN


In [8]:
# 3.2. Tratamento de valores faltantes
# TODO: definir estratégia (drop, média/mediana, categoria 'desconhecido', etc.)
# Exemplo simples (apenas sugestão; você pode mudar):
titanic['Embarked'] = titanic['Embarked'].fillna(titanic['Embarked'].mode()[0])
titanic['Age'] = titanic['Age'].fillna(titanic['Age'].median())
titanic['Fare'] = titanic['Fare'].fillna(titanic['Fare'].median())

# Verificar novamente
titanic.isna().sum()


,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,0
SibSp,0
Parch,0
Ticket,0
Fare,0


In [9]:
# 4. Criação de atributos categóricos (discretização)
# 4.1. Survived em formato categórico
titanic['Survived_cat'] = titanic['Survived'].map({0: 'Survived=nao', 1: 'Survived=sim'})

# 4.2. Discretização de Idade
# TODO: ajustar faixas conforme desejado
bins_age = [0, 12, 18, 35, 60, np.inf]
labels_age = ['crianca', 'adolescente', 'jovem_adulto', 'adulto', 'idoso']
titanic['Age_cat'] = pd.cut(titanic['Age'], bins=bins_age, labels=labels_age, right=False)

# 4.3. Discretização de Fare (tarifa)
# Exemplo via quartis (você pode propor outra regra)
titanic['Fare_cat'] = pd.qcut(titanic['Fare'], q=4, labels=['baixa', 'media', 'alta', 'muito_alta'])

# 4.4. Atributos derivados de família
titanic['FamilySize'] = titanic['SibSp'] + titanic['Parch']
titanic['Viajava_sozinho'] = np.where(titanic['FamilySize'] == 0, 'sozinho', 'com_familia')

# 4.5. Selecionar apenas variáveis categóricas que serão usadas nas regras
cols_categoricas = [
    'Survived_cat',
    'Pclass',
    'Sex',
    'Age_cat',
    'Fare_cat',
    'Embarked',
    'Viajava_sozinho'
]

dados_cat = titanic[cols_categoricas].copy()
dados_cat.head()


,Survived_cat,Pclass,Sex,Age_cat,Fare_cat,Embarked,Viajava_sozinho
0,Survived=nao,3,male,jovem_adulto,baixa,S,com_familia
1,Survived=sim,1,female,adulto,muito_alta,C,com_familia
2,Survived=sim,3,female,jovem_adulto,media,S,sozinho
3,Survived=sim,1,female,adulto,muito_alta,S,com_familia
4,Survived=nao,3,male,adulto,media,S,sozinho


In [10]:
import pandas as pd

# 5. One-hot encoding (cada item como coluna 0/1)
# get_dummies já cria colunas do tipo 'Sex_male', 'Sex_female', etc.

# Convert 'Pclass' to object/string type before one-hot encoding
dados_cat['Pclass'] = dados_cat['Pclass'].astype(str)
dados_onehot = pd.get_dummies(dados_cat)

dados_onehot.head()

,Survived_cat_Survived=nao,Survived_cat_Survived=sim,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Age_cat_crianca,Age_cat_adolescente,Age_cat_jovem_adulto,Age_cat_adulto,Age_cat_idoso,Fare_cat_baixa,Fare_cat_media,Fare_cat_alta,Fare_cat_muito_alta,Embarked_C,Embarked_Q,Embarked_S,Viajava_sozinho_com_familia,Viajava_sozinho_sozinho
0,True,False,False,False,True,False,True,False,False,True,False,False,True,False,False,False,False,False,True,True,False
1,False,True,True,False,False,True,False,False,False,False,True,False,False,False,False,True,True,False,False,True,False
2,False,True,False,False,True,True,False,False,False,True,False,False,False,True,False,False,False,False,True,False,True
3,False,True,True,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,True,True,False
4,True,False,False,False,True,False,True,False,False,False,True,False,False,True,False,False,False,False,True,False,True


In [11]:
# 6. Descobrir itemsets frequentes – Cenário A (suporte mais alto)
min_sup_A = 0.01  # por exemplo, 10% das transações

frequent_itemsets_A = apriori(
    dados_onehot,
    min_support=min_sup_A,
    use_colnames=True
)

frequent_itemsets_A.sort_values('support', ascending=False).head()

,support,itemsets
18,0.725,(Embarked_S)
6,0.648,(Sex_male)
0,0.616,(Survived_cat_Survived=nao)
9,0.609,(Age_cat_jovem_adulto)
20,0.603,(Viajava_sozinho_sozinho)


In [12]:
# 7. Regras de associação para Cenário A
min_conf = 0.6  # confiança mínima inicial (ajustar se gerar poucas/muitas regras)

regras_A = association_rules(
    frequent_itemsets_A,
    metric='confidence',
    min_threshold=min_conf
)

# Ordenar por lift para inspecionar
regras_A.sort_values('lift', ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
6632,"(Viajava_sozinho_sozinho, Survived_cat_Survive...","(Fare_cat_baixa, Sex_female)",0.020,0.048,0.018,0.889,18.419,1.0,0.017,8.566,0.965,0.356,0.883,0.630
6647,"(Pclass_3, Embarked_Q, Viajava_sozinho_sozinho...","(Fare_cat_baixa, Sex_female, Age_cat_jovem_adu...",0.024,0.042,0.018,0.762,18.347,1.0,0.017,4.026,0.968,0.381,0.752,0.597
5892,"(Pclass_3, Embarked_Q, Viajava_sozinho_sozinho...","(Fare_cat_baixa, Sex_female)",0.024,0.048,0.020,0.857,17.761,1.0,0.019,6.662,0.966,0.391,0.850,0.638
6651,"(Embarked_Q, Viajava_sozinho_sozinho, Survived...","(Fare_cat_baixa, Sex_female, Age_cat_jovem_adu...",0.026,0.042,0.018,0.696,16.752,1.0,0.017,3.149,0.965,0.364,0.682,0.564
5980,"(Embarked_Q, Viajava_sozinho_sozinho, Survived...","(Fare_cat_baixa, Sex_female, Age_cat_jovem_adu...",0.026,0.042,0.018,0.696,16.752,1.0,0.017,3.149,0.965,0.364,0.682,0.564


In [13]:
# 8. Filtrar regras relacionadas à sobrevivência – Cenário A (Consequente Único)

# Filtrar regras onde o consequente é EXATAMENTE 'Survived_cat_Survived=sim'
regras_A_sim = regras_A[
    regras_A['consequents'].apply(lambda x: list(x) == ['Survived_cat_Survived=sim'])
].copy()

# Filtrar regras onde o consequente é EXATAMENTE 'Survived_cat_Survived=nao'
regras_A_nao = regras_A[
    regras_A['consequents'].apply(lambda x: list(x) == ['Survived_cat_Survived=nao'])
].copy()

print('Regras com Survived=sim (Cenário A - Consequente único):')
regras_A_sim.sort_values('lift', ascending=False).head(10)

Regras com Survived=sim (Cenário A - Consequente único):


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
5788,"(Viajava_sozinho_com_familia, Age_cat_crianca,...",(Survived_cat_Survived=sim),0.012,0.384,0.012,1.0,2.605,1.0,0.008,inf,0.624,0.032,1.0,0.516
5514,"(Sex_female, Viajava_sozinho_com_familia, Pcla...",(Survived_cat_Survived=sim),0.011,0.384,0.011,1.0,2.605,1.0,0.007,inf,0.623,0.029,1.0,0.515
5525,"(Sex_female, Age_cat_adulto, Pclass_1, Embarke...",(Survived_cat_Survived=sim),0.020,0.384,0.020,1.0,2.605,1.0,0.012,inf,0.629,0.053,1.0,0.526
1270,"(Viajava_sozinho_com_familia, Age_cat_crianca,...",(Survived_cat_Survived=sim),0.019,0.384,0.019,1.0,2.605,1.0,0.012,inf,0.628,0.050,1.0,0.525
1264,"(Age_cat_crianca, Fare_cat_alta, Pclass_2)",(Survived_cat_Survived=sim),0.012,0.384,0.012,1.0,2.605,1.0,0.008,inf,0.624,0.032,1.0,0.516
1267,"(Age_cat_crianca, Embarked_S, Pclass_2)",(Survived_cat_Survived=sim),0.017,0.384,0.017,1.0,2.605,1.0,0.010,inf,0.627,0.044,1.0,0.522
1262,"(Age_cat_crianca, Sex_male, Pclass_2)",(Survived_cat_Survived=sim),0.010,0.384,0.010,1.0,2.605,1.0,0.006,inf,0.622,0.026,1.0,0.513
1251,"(Sex_female, Fare_cat_muito_alta, Pclass_2)",(Survived_cat_Survived=sim),0.010,0.384,0.010,1.0,2.605,1.0,0.006,inf,0.622,0.026,1.0,0.513
5545,"(Sex_female, Age_cat_adulto, Viajava_sozinho_c...",(Survived_cat_Survived=sim),0.030,0.384,0.030,1.0,2.605,1.0,0.019,inf,0.635,0.079,1.0,0.539
5538,"(Sex_female, Age_cat_adulto, Pclass_1, Fare_ca...",(Survived_cat_Survived=sim),0.020,0.384,0.020,1.0,2.605,1.0,0.012,inf,0.629,0.053,1.0,0.526


In [14]:
# 9. Cenário B – suporte mais baixo
min_sup_B = 0.05  # por exemplo, 5%
min_conf = 0.6 # Definindo min_conf explicitamente aqui

frequent_itemsets_B = apriori(
    dados_onehot,
    min_support=min_sup_B,
    use_colnames=True
)

regras_B = association_rules(
    frequent_itemsets_B,
    metric='confidence',
    min_threshold=min_conf
)

regras_B_sim = regras_B[regras_B['consequents'].astype(str).str.contains('Survived=sim')].copy()
regras_B_nao = regras_B[regras_B['consequents'].astype(str).str.contains('Survived=nao')].copy()

print('Top 10 regras Survived=sim (Cenário B):')
regras_B_sim.sort_values('lift', ascending=False).head(10)

Top 10 regras Survived=sim (Cenário B):


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
1315,"(Sex_female, Viajava_sozinho_com_familia, Pcla...","(Fare_cat_muito_alta, Survived_cat_Survived=sim)",0.067,0.145,0.064,0.950,6.562,1.0,0.054,17.104,0.909,0.432,0.942,0.696
698,"(Pclass_1, Sex_female)","(Fare_cat_muito_alta, Survived_cat_Survived=sim)",0.105,0.145,0.094,0.894,6.172,1.0,0.079,8.039,0.937,0.604,0.876,0.772
1322,"(Sex_female, Pclass_1)","(Viajava_sozinho_com_familia, Fare_cat_muito_a...",0.105,0.102,0.064,0.606,5.937,1.0,0.053,2.281,0.930,0.445,0.562,0.616
727,"(Pclass_1, Viajava_sozinho_com_familia)","(Fare_cat_muito_alta, Survived_cat_Survived=sim)",0.120,0.145,0.086,0.720,4.970,1.0,0.069,3.050,0.908,0.484,0.672,0.658
700,"(Sex_female, Fare_cat_muito_alta)","(Pclass_1, Survived_cat_Survived=sim)",0.125,0.153,0.094,0.757,4.958,1.0,0.075,3.484,0.912,0.515,0.713,0.687
719,"(Fare_cat_muito_alta, Embarked_C)","(Pclass_1, Survived_cat_Survived=sim)",0.083,0.153,0.058,0.703,4.604,1.0,0.046,2.850,0.854,0.329,0.649,0.543
1316,"(Sex_female, Viajava_sozinho_com_familia, Fare...","(Pclass_1, Survived_cat_Survived=sim)",0.093,0.153,0.064,0.687,4.499,1.0,0.050,2.705,0.858,0.352,0.630,0.553
718,"(Pclass_1, Embarked_C)","(Fare_cat_muito_alta, Survived_cat_Survived=sim)",0.095,0.145,0.058,0.612,4.225,1.0,0.045,2.203,0.844,0.321,0.546,0.507
714,"(Fare_cat_muito_alta, Age_cat_adulto)","(Pclass_1, Survived_cat_Survived=sim)",0.086,0.153,0.054,0.623,4.084,1.0,0.041,2.250,0.827,0.291,0.556,0.488
1318,"(Sex_female, Pclass_1, Fare_cat_muito_alta)","(Viajava_sozinho_com_familia, Survived_cat_Sur...",0.097,0.201,0.064,0.663,3.299,1.0,0.045,2.370,0.771,0.274,0.578,0.491


In [15]:
# 10. Exploração para buscar possíveis regras redundantes
# Dica: ordenar por support e confidence e inspecionar regras com antecedentes parecidos
regras_B_sim.sort_values(['support', 'confidence'], ascending=False).head(20)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
14,(Sex_female),(Survived_cat_Survived=sim),0.352,0.384,0.262,0.742,1.933,1.0,0.126,2.389,0.745,0.551,0.581,0.712
194,"(Sex_female, Embarked_S)",(Survived_cat_Survived=sim),0.230,0.384,0.159,0.693,1.805,1.0,0.071,2.005,0.579,0.351,0.501,0.554
13,(Pclass_1),(Survived_cat_Survived=sim),0.242,0.384,0.153,0.630,1.640,1.0,0.060,1.664,0.515,0.322,0.399,0.514
197,"(Viajava_sozinho_com_familia, Sex_female)",(Survived_cat_Survived=sim),0.211,0.384,0.150,0.713,1.857,1.0,0.069,2.145,0.585,0.338,0.534,0.552
184,"(Age_cat_jovem_adulto, Sex_female)",(Survived_cat_Survived=sim),0.200,0.384,0.146,0.730,1.903,1.0,0.069,2.285,0.593,0.333,0.562,0.555
172,"(Pclass_1, Fare_cat_muito_alta)",(Survived_cat_Survived=sim),0.178,0.384,0.122,0.686,1.786,1.0,0.054,1.959,0.536,0.278,0.490,0.502
199,"(Sex_female, Viajava_sozinho_sozinho)",(Survived_cat_Survived=sim),0.141,0.384,0.111,0.786,2.047,1.0,0.057,2.875,0.596,0.268,0.652,0.538
190,"(Sex_female, Fare_cat_muito_alta)",(Survived_cat_Survived=sim),0.125,0.384,0.107,0.856,2.230,1.0,0.059,4.275,0.630,0.265,0.766,0.567
167,"(Pclass_1, Sex_female)",(Survived_cat_Survived=sim),0.105,0.384,0.102,0.968,2.522,1.0,0.062,19.306,0.675,0.264,0.948,0.617
757,"(Viajava_sozinho_com_familia, Embarked_S, Sex_...",(Survived_cat_Survived=sim),0.147,0.384,0.098,0.664,1.730,1.0,0.041,1.834,0.495,0.225,0.455,0.459


## 11. Dado o código acima, realize as seguintes tarefas

Use o código acima como ponto de partida e responda às questões abaixo **diretamente neste notebook** (em novas células Markdown ou de código, quando fizer sentido).

### Tarefa 1 – Exploração de parâmetros
1. Altere os valores de `min_sup_A`, `min_sup_B` e `min_conf` e avalie o impacto:
   - Quantas regras são geradas em cada cenário?
   - Elas ficam mais gerais ou mais específicas?
2. Registre suas observações em um pequeno texto comparando os diferentes conjuntos de parâmetros.

### Tarefa 2 – Perfis de alta e baixa sobrevivência
1. Escolha **3 regras** com consequente `Survived=sim` e `lift > 1`.
   - Para cada uma, escreva uma interpretação em linguagem natural, explicando o perfil de passageiro descrito.
2. Escolha **3 regras** com consequente `Survived=nao` e `lift > 1`.
   - Para cada uma, descreva o perfil de passageiros com menor chance de sobrevivência.

### Tarefa 3 – Regras redundantes
1. Usando as tabelas de regras (especialmente de `regras_B_sim` e `regras_B_nao`),
   identifique **pelo menos 2 pares de regras** em que uma seja redundante em relação à outra
   (mesmo suporte e confiança, mas antecedente/consequente contendo mais itens).
2. Explique por que você considera cada par de regras redundantes, usando o conceito de **superconjunto**.

### Tarefa 4 – Discussão crítica
Responda em 10–15 linhas:
1. Os padrões encontrados reforçam a ideia histórica de que *"mulheres e crianças primeiro"*?
2. Há evidências de desigualdade por classe social (`Pclass`) nas regras encontradas?
3. Quais são as principais limitações dessa análise com regras de associação neste dataset
   (por exemplo, viés dos dados, discretizações, variáveis ausentes etc.)?

---
**Entrega sugerida:**

- Notebook preenchido com o código ajustado e as respostas às tarefas nas células Markdown.



In [16]:
# === Tarefa 1 – Exploração de parâmetros ===

# Cenário original do notebook
min_sup_A = 0.01
min_sup_B = 0.05
min_conf = 0.6

frequent_itemsets_A = apriori(dados_onehot, min_support=min_sup_A, use_colnames=True)
regras_A = association_rules(frequent_itemsets_A, metric='confidence', min_threshold=min_conf)

frequent_itemsets_B = apriori(dados_onehot, min_support=min_sup_B, use_colnames=True)
regras_B = association_rules(frequent_itemsets_B, metric='confidence', min_threshold=min_conf)

print('Cenário original:')
print(f'  Cenário A -> {len(regras_A)} regras')
print(f'  Cenário B -> {len(regras_B)} regras')

# Novo conjunto de parâmetros mais restritivo
min_sup_A_2 = 0.02
min_sup_B_2 = 0.08
min_conf_2 = 0.8

frequent_itemsets_A_2 = apriori(dados_onehot, min_support=min_sup_A_2, use_colnames=True)
regras_A_2 = association_rules(frequent_itemsets_A_2, metric='confidence', min_threshold=min_conf_2)

frequent_itemsets_B_2 = apriori(dados_onehot, min_support=min_sup_B_2, use_colnames=True)
regras_B_2 = association_rules(frequent_itemsets_B_2, metric='confidence', min_threshold=min_conf_2)

print('\nCenário 2 (suporte/confiança maiores):')
print(f'  Cenário A -> {len(regras_A_2)} regras')
print(f'  Cenário B -> {len(regras_B_2)} regras')

Cenário original:
  Cenário A -> 6663 regras
  Cenário B -> 1723 regras

Cenário 2 (suporte/confiança maiores):
  Cenário A -> 1897 regras
  Cenário B -> 374 regras


## 11. Resolução das tarefas

### Tarefa 1 – Exploração de parâmetros

Usando o código acima, obtive os seguintes resultados:

- **Cenário original** (`min_sup_A = 0.01`, `min_sup_B = 0.05`, `min_conf = 0.6`):
  - Cenário A → **6.663 regras**
  - Cenário B → **1.723 regras**
- **Cenário 2 mais restritivo** (`min_sup_A = 0.02`, `min_sup_B = 0.08`, `min_conf = 0.8`):
  - Cenário A → **1.897 regras**
  - Cenário B → **374 regras**

**Observações:**
- Quando **aumentamos o suporte mínimo e a confiança mínima**, o número total de regras cai bastante.
- Com limiares mais altos, **sobram apenas padrões muito frequentes**, geralmente com antecedentes mais curtos (regras mais **gerais**).
- Com limiares mais baixos, aparecem muitas combinações raras, com antecedentes maiores, que geram regras mais **específicas**, porém com suporte pequeno.

---

### Tarefa 2 – Perfis de alta e baixa sobrevivência

As interpretações abaixo foram feitas a partir das regras do **Cenário B** (`min_sup_B = 0.05`, `min_conf = 0.6`), ordenadas por *lift*.

#### (a) Três regras com consequente `Survived=sim` e `lift > 1`

1. **Antecedente:** `Sex_female` & `Pclass_1`  →  **Consequente:** `Survived=sim`  
   - Interpretação: mulheres que viajavam na **1ª classe** têm uma probabilidade de sobrevivência muito alta (conf. ≈ 97%),
     mais de **2,5 vezes** maior do que a média geral (lift > 2,5). Representa o perfil de passageira rica e prioritária no resgate.

2. **Antecedente:** `Sex_female` & `Pclass_2`  →  **Consequente:** `Survived=sim`  
   - Interpretação: mulheres na **2ª classe** também apresentam grande chance de sobreviver (conf. ≈ 92% e lift ≈ 2,4).
     Mesmo não sendo as mais ricas, ainda recebem forte prioridade nas embarcações de salvamento.

3. **Antecedente:** `Sex_female` & `Fare_cat_muito_alta` & `Pclass_1`  →  **Consequente:** `Survived=sim`  
   - Interpretação: passageiras mulheres que pagaram tarifa **muito alta** na **1ª classe** quase sempre sobrevivem
     (conf. ≈ 98% e lift ≈ 2,5). É o perfil extremo de passageiras mais ricas e com maior acesso aos botes.

#### (b) Três regras com consequente `Survived=nao` e `lift > 1`

1. **Antecedente:** `Fare_cat_baixa` & `Sex_male`  →  **Consequente:** `Survived=nao`  
   - Interpretação: homens que pagaram **tarifas baixas** (mais pobres) têm alta chance de morrer (conf. ≈ 92%, lift ≈ 1,5).
     São, em geral, passageiros da 3ª classe em posições desfavorecidas no navio.

2. **Antecedente:** `Fare_cat_baixa` & `Embarked_S` & `Sex_male`  →  **Consequente:** `Survived=nao`  
   - Interpretação: homens embarcados em **Southampton**, com **baixa tarifa**, apresentam probabilidade ainda maior de não sobreviver
     (conf. ≈ 93%). Mostra como um grupo grande, de homens trabalhadores embarcados nesse porto, ficou entre os mais vulneráveis.

3. **Antecedente:** `Age_cat_adulto` & `Pclass_3` & `Sex_male`  →  **Consequente:** `Survived=nao`  
   - Interpretação: **homens adultos da 3ª classe** aparecem com probabilidade muito alta de morte (conf. ≈ 94%, lift ≈ 1,52).
     É o perfil clássico de quem ficou para trás: homem, pobre, adulto e sem prioridade no resgate.

---

### Tarefa 3 – Regras redundantes

Usando principalmente as tabelas `regras_B_sim` e `regras_B_nao`, podemos identificar pares de regras em que
uma é praticamente um **superconjunto** da outra, com suporte e confiança muito parecidos.

**Par 1 – Sobrevivência (`Survived=sim`)**
- Regra 1: `Sex_female` & `Pclass_1`  →  `Survived=sim`
- Regra 2: `Sex_female` & `Pclass_1` & `Fare_cat_muito_alta`  →  `Survived=sim`

A Regra 2 adiciona apenas a condição de tarifa **muito alta**, mas o ganho em lift/confiança é pequeno.
Como o consequente é o mesmo e o perfil básico "mulher de 1ª classe" já explica bem a sobrevivência,
a regra mais específica se torna **redundante**, pois é um superconjunto da primeira.

**Par 2 – Não sobrevivência (`Survived=nao`)**
- Regra 3: `Fare_cat_baixa` & `Sex_male`  →  `Survived=nao`
- Regra 4: `Fare_cat_baixa` & `Sex_male` & `Viajava_sozinho_sozinho` & `Embarked_S`  →  `Survived=nao`

A Regra 4 apenas detalha ainda mais o mesmo grupo (homem, pobre), adicionando que viaja **sozinho** e que embarcou em **S**.
Como suporte e confiança são muito próximos e o consequente é igual, ela é redundante em relação à Regra 3.
Nesses casos, costuma-se preferir a regra com **antecedente menor**, por ser mais simples e fácil de interpretar.

---

### Tarefa 4 – Discussão crítica (10–15 linhas)

1. **Mulheres e crianças primeiro?**  
   As regras com consequente `Survived=sim` mostram forte efeito de gênero e, em cenários com suporte mais baixo,
   também aparecem combinações envolvendo crianças com *lift* elevado. Isso reforça, em parte, a narrativa histórica de
   "mulheres e crianças primeiro", sobretudo para as classes mais altas. No entanto, o efeito é bem mais claro para
   **mulheres adultas de 1ª e 2ª classe** do que para todas as crianças em geral.

2. **Desigualdade por classe social (`Pclass`)**  
   Há evidências claras de desigualdade: regras com `Pclass_1` e `Pclass_2` associadas a `Sex_female` levam a alta
   probabilidade de sobrevivência, enquanto regras com `Pclass_3`, `Fare_cat_baixa` e `Sex_male` estão ligadas a
   `Survived=nao` com grande confiança. Ou seja, além do gênero, a **classe social** (posição no navio, tipo de cabine,
   proximidade dos botes) influencia fortemente quem teve acesso ao resgate.

3. **Limitações da análise com regras de associação**  
   - As regras capturam **correlações**, não causalidade: não podemos concluir que ser homem da 3ª classe "causa" a morte,
     apenas que está fortemente associado a ela neste conjunto de dados.
   - A análise depende de várias escolhas arbitrárias: discretização de idade e tarifa, seleção de variáveis, valores de
     suporte/confiança etc., que podem mudar bastante o conjunto de regras encontradas.
   - O dataset do Titanic é relativamente pequeno e histórico; os padrões podem refletir viés de registro ou simplificações
     (por exemplo, ausência de informações completas sobre raça, profissão, localização exata na embarcação).
   - Regras com suporte muito baixo tendem a ser pouco estáveis (podem ser fruto de acaso) e, por isso, é necessário
     complementá-las com outras técnicas estatísticas e com a interpretação histórica do contexto.
